# 24-1 기계학습 HW#8 : YOLOv8 tracking with custom dataset
## 작성자 : 2019038074 이우창

<br>

참고 자료: https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb

## 드라이브 연결 및 ultralytics 설치

In [1]:
# prompt: 드라이브에 연결

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.26 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.0/78.2 GB disk)


## Roboflow 데이터셋 다운로드

In [3]:
!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="<api_key>")
project = rf.workspace("studio1122").project("yolo-tracking")
version = project.version(5)
dataset = version.download("yolov8")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.4 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.26, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to yolo-tracking-5 in yolov8:: 100%|██████████| 1666/1666 [00:00<00:00, 3077.38it/s]


## 전이 학습


In [4]:
# data.yaml의 경로 수정 (코랩에서의 파일 경로와 중복되는 부분이 존재)
!sed -i "s|train: yolo-tracking-5/|train: |g" ./yolo-tracking-5/data.yaml
!sed -i "s|val: yolo-tracking-5/|val: |g" ./yolo-tracking-5/data.yaml

In [5]:
!yolo task=detect mode=train model=yolov8m.pt data=/content/yolo-tracking-5/data.yaml epochs=25 imgsz=800 plots=True

100% 49.7M/49.7M [00:00<00:00, 246MB/s]
Ultralytics YOLOv8.2.26 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/yolo-tracking-5/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cro

## 검증 데이터 평가

In [6]:
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data=/content/yolo-tracking-5/data.yaml

Ultralytics YOLOv8.2.26 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /content/yolo-tracking-5/valid/labels.cache... 64 images, 0 backgrounds, 0 corrupt: 100% 64/64 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 4/4 [00:13<00:00,  3.41s/it]
                   all         64        130      0.911      0.878      0.913      0.672
                magpie         23         32      0.852      0.906       0.91      0.639
            no-parking         21         33      0.967      0.899      0.934      0.746
                pigeon         20         65      0.915       0.83      0.894      0.631
Speed: 18.8ms preprocess, 35.6ms inference, 0.4ms loss, 69.1ms postprocess per image
Results saved to runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [7]:
!cp -r /content/runs/* "/content/drive/MyDrive/Colab Notebooks/YOLO tracking/runs_version5m"

## Tracking 및 동영상으로 저장

In [8]:
import os

def get_file_list(directory):
    return os.listdir(directory)

directory = '/content/drive/MyDrive/Colab Notebooks/YOLO tracking/videos'
file_list = get_file_list(directory)
print(file_list)

['magpie_fly.mp4', 'magpie_walking.mp4', 'pigeon_walking.mp4', 'pigeon_cluster.mp4', 'magpie_walking2.mp4', 'magpie_on_tree.mp4', 'magpie_solo.mp4', 'magpie_walking3.mp4', 'magpie_walking4.mp4', 'pigeon_cluster2.mp4', 'pigeon_walking2.mp4', 'pigeon_walking3.mp4', 'pigeon_walking4.mp4', 'no_parking1.mp4', 'no_parking2.mp4']


In [9]:
from collections import defaultdict

import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("/content/runs/detect/train/weights/best.pt")

def get_file_list(directory):
    return os.listdir(directory)

directory = '/content/drive/MyDrive/Colab Notebooks/YOLO tracking/videos'
file_list = get_file_list(directory)

for filename in file_list:
  # Open the video file
  video_path = "/content/drive/MyDrive/Colab Notebooks/YOLO tracking/videos/" + filename
  cap = cv2.VideoCapture(video_path)

  # Store the track history
  track_history = defaultdict(lambda: [])

  # Retrieve video properties: width, height, and frames per second
  w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

  # Initialize video writer to save the output video with the specified properties
  out = cv2.VideoWriter("/content/drive/MyDrive/Colab Notebooks/YOLO tracking/videos_version5m/" +
                        filename.split('.')[0] + ".avi", cv2.VideoWriter_fourcc(*"MJPG"), fps, (w, h))

  # Loop through the video frames
  while cap.isOpened():
      # Read a frame from the video
      success, frame = cap.read()

      if success:
          # Run YOLOv8 tracking on the frame, persisting tracks between frames
          results = model.track(frame, persist=True)

          # Get the boxes and track IDs
          if results[0].boxes is not None and results[0].boxes.id is not None:
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot(line_width=5, font_size=14, pil=True)

            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 90 tracks for 90 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=5)

          else:
            annotated_frame = frame

          # Write the annotated frame to the output video
          out.write(annotated_frame)

          # Display the annotated frame
          #cv2.imshow("YOLOv8 Tracking", annotated_frame)

          # Break the loop if 'q' is pressed
          if cv2.waitKey(1) & 0xFF == ord("q"):
              break
      else:
          # Break the loop if the end of the video is reached
          break

  # Release the video capture object and close the display window
  cap.release()
  cv2.destroyAllWindows()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0: 480x800 4 magpies, 35.9ms
Speed: 4.3ms preprocess, 35.9ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 4 magpies, 35.9ms
Speed: 4.4ms preprocess, 35.9ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 4 magpies, 35.9ms
Speed: 4.1ms preprocess, 35.9ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 4 magpies, 36.0ms
Speed: 5.0ms preprocess, 36.0ms inference, 5.9ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 4 magpies, 35.9ms
Speed: 5.4ms preprocess, 35.9ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 4 magpies, 36.0ms
Speed: 8.0ms preprocess, 36.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 4 magpies, 35.9ms
Speed: 6.1ms preprocess, 35.9ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 4 magpies, 35.9ms
Speed: 5.0ms preprocess, 35.9ms inference, 1.4